# Feature Engineering — Bureau Table (Relational Data)

## Objective
* Create customer-level credit history features from `bureau.csv`
* Aggregate by `SK_ID_CURR` and merge into the application table
* Re-train the model and compare AUC vs the baseline

## Why this matters
* Application data describes the applicant.
* Bureau data describes historical credit behavior, which is often the strongest driver of default risk.


In [1]:
import numpy as np
import pandas as pd
import re

from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier

In [2]:
train = pd.read_csv("../data/application_train.csv")
test  = pd.read_csv("../data/application_test.csv")

bureau = pd.read_csv("../data/bureau.csv")

print("train:", train.shape, "test:", test.shape, "bureau:", bureau.shape)
train.head()

train: (307511, 122) test: (48744, 121) bureau: (1716428, 17)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


# 1. Bureau Table — Quick Check

## What is bureau?
* Contains client credit history information from other financial institutions.
* Key identifiers:
  * `SK_ID_CURR`: client id (join key to application)
  * `SK_ID_BUREAU`: bureau record id (one client → many records)

## Goal today
* Build aggregated features per `SK_ID_CURR`

In [3]:
bureau[["SK_ID_CURR", "SK_ID_BUREAU"]].nunique()
bureau.isnull().mean().sort_values(ascending=False).head(15) * 100

AMT_ANNUITY               71.473490
AMT_CREDIT_MAX_OVERDUE    65.513264
DAYS_ENDDATE_FACT         36.916958
AMT_CREDIT_SUM_LIMIT      34.477415
AMT_CREDIT_SUM_DEBT       15.011932
DAYS_CREDIT_ENDDATE        6.149573
AMT_CREDIT_SUM             0.000757
CREDIT_ACTIVE              0.000000
CREDIT_CURRENCY            0.000000
DAYS_CREDIT                0.000000
CREDIT_DAY_OVERDUE         0.000000
SK_ID_BUREAU               0.000000
CNT_CREDIT_PROLONG         0.000000
AMT_CREDIT_SUM_OVERDUE     0.000000
CREDIT_TYPE                0.000000
dtype: float64

# 2. Feature Engineering Plan (Bureau)

## We will create:
* *Counts*
  * total bureau records per client
  * number of active credits
  * number of closed credits
* *Aggregates (numeric)*
  * mean / max / min of credit amounts, overdue amounts, etc.
* *Ratios / flags (domain features)*
  * active_ratio = active_count / total_count
  * overdue_flag and overdue_rate


In [5]:
def agg_bureau(bureau_df: pd.DataFrame) -> pd.DataFrame:
    b = bureau_df.copy()

    # --- Basic flags ---
    b["CREDIT_ACTIVE_IS_ACTIVE"] = (b["CREDIT_ACTIVE"] == "Active").astype(int)
    b["CREDIT_ACTIVE_IS_CLOSED"] = (b["CREDIT_ACTIVE"] == "Closed").astype(int)

    # Overdue signals (some columns can be missing)
    if "AMT_CREDIT_SUM_OVERDUE" in b.columns:
        b["OVERDUE_FLAG"] = (b["AMT_CREDIT_SUM_OVERDUE"].fillna(0) > 0).astype(int)
    else:
        b["OVERDUE_FLAG"] = 0

    # --- Aggregations ---
    agg_dict = {
        "SK_ID_BUREAU": ["count"],
        "CREDIT_ACTIVE_IS_ACTIVE": ["sum", "mean"],
        "CREDIT_ACTIVE_IS_CLOSED": ["sum", "mean"],
        "OVERDUE_FLAG": ["sum", "mean"],
    }

    # Numeric columns worth aggregating
    numeric_candidates = [
        "DAYS_CREDIT",
        "CREDIT_DAY_OVERDUE",
        "DAYS_CREDIT_ENDDATE",
        "DAYS_ENDDATE_FACT",
        "AMT_CREDIT_MAX_OVERDUE",
        "AMT_CREDIT_SUM",
        "AMT_CREDIT_SUM_DEBT",
        "AMT_CREDIT_SUM_LIMIT",
        "AMT_CREDIT_SUM_OVERDUE",
        "AMT_ANNUITY",
    ]

    for col in numeric_candidates:
        if col in b.columns:
            agg_dict[col] = ["mean", "max", "min", "sum"]

    buro_agg = b.groupby("SK_ID_CURR").agg(agg_dict)

    # Flatten column names
    buro_agg.columns = [
        f"BURO_{c[0]}_{c[1].upper()}" for c in buro_agg.columns.to_flat_index()
    ]
    buro_agg = buro_agg.reset_index()

    # --- Extra ratios ---
    buro_agg["BURO_ACTIVE_RATIO"] = (
        buro_agg["BURO_CREDIT_ACTIVE_IS_ACTIVE_SUM"] /
        buro_agg["BURO_SK_ID_BUREAU_COUNT"].replace(0, np.nan)
    )
    buro_agg["BURO_CLOSED_RATIO"] = (
        buro_agg["BURO_CREDIT_ACTIVE_IS_CLOSED_SUM"] /
        buro_agg["BURO_SK_ID_BUREAU_COUNT"].replace(0, np.nan)
    )
    buro_agg["BURO_OVERDUE_RATE"] = (
        buro_agg["BURO_OVERDUE_FLAG_MEAN"]
    )

    return buro_agg

In [6]:
bureau_features = agg_bureau(bureau)
bureau_features.head()

,SK_ID_CURR,BURO_SK_ID_BUREAU_COUNT,BURO_CREDIT_ACTIVE_IS_ACTIVE_SUM,BURO_CREDIT_ACTIVE_IS_ACTIVE_MEAN,BURO_CREDIT_ACTIVE_IS_CLOSED_SUM,BURO_CREDIT_ACTIVE_IS_CLOSED_MEAN,BURO_OVERDUE_FLAG_SUM,BURO_OVERDUE_FLAG_MEAN,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_MAX,...,BURO_AMT_CREDIT_SUM_OVERDUE_MAX,BURO_AMT_CREDIT_SUM_OVERDUE_MIN,BURO_AMT_CREDIT_SUM_OVERDUE_SUM,BURO_AMT_ANNUITY_MEAN,BURO_AMT_ANNUITY_MAX,BURO_AMT_ANNUITY_MIN,BURO_AMT_ANNUITY_SUM,BURO_ACTIVE_RATIO,BURO_CLOSED_RATIO,BURO_OVERDUE_RATE
0,100001,7,3,0.428571,4,0.571429,0,0.0,-735.000000,-49,...,0.0,0.0,0.0,3545.357143,10822.5,0.0,24817.5,0.428571,0.571429,0.0
1,100002,8,2,0.250000,6,0.750000,0,0.0,-874.000000,-103,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.250000,0.750000,0.0
2,100003,4,1,0.250000,3,0.750000,0,0.0,-1400.750000,-606,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.250000,0.750000,0.0
3,100004,2,0,0.000000,2,1.000000,0,0.0,-867.000000,-408,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.000000,1.000000,0.0
4,100005,3,2,0.666667,1,0.333333,0,0.0,-190.666667,-62,...,0.0,0.0,0.0,1420.500000,4261.5,0.0,4261.5,0.666667,0.333333,0.0


# 3. Merge Bureau Features into Application Data

## Approach
* Left join on `SK_ID_CURR`
* Missing bureau features indicate a client has no bureau records
* We will impute missing values during preprocessing

In [7]:
train_fe = train.merge(bureau_features, on="SK_ID_CURR", how="left")
test_fe  = test.merge(bureau_features, on="SK_ID_CURR", how="left")

print("train_fe:", train_fe.shape, "test_fe:", test_fe.shape)

train_fe: (307511, 172) test_fe: (48744, 171)


# 4. Model Training — Application + Bureau Features

## Preprocessing
* Numeric: median impute
* Categorical: most frequent impute + one-hot encoding
* Feature name sanitation for LightGBM compatibility

## Validation
* 5-fold Stratified CV
* Metric: AUC

In [8]:
def clean_feature_names(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"[^A-Za-z0-9_]+", "_", str(c)) for c in df.columns]
    return df

In [9]:
y = train_fe["TARGET"]
X = train_fe.drop(columns=["TARGET", "SK_ID_CURR"])
test_ids = test_fe["SK_ID_CURR"]
X_test = test_fe.drop(columns=["SK_ID_CURR"])

num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

In [10]:
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

# numeric
X_num = pd.DataFrame(num_imputer.fit_transform(X[num_cols]), columns=num_cols, index=X.index)
T_num = pd.DataFrame(num_imputer.transform(X_test[num_cols]), columns=num_cols, index=X_test.index)

# categorical
X_cat = pd.DataFrame(cat_imputer.fit_transform(X[cat_cols]), columns=cat_cols, index=X.index)
T_cat = pd.DataFrame(cat_imputer.transform(X_test[cat_cols]), columns=cat_cols, index=X_test.index)

X_cat_ohe = pd.DataFrame(ohe.fit_transform(X_cat), columns=ohe.get_feature_names_out(cat_cols), index=X.index)
T_cat_ohe = pd.DataFrame(ohe.transform(T_cat), columns=ohe.get_feature_names_out(cat_cols), index=X_test.index)

X_processed = pd.concat([X_num, X_cat_ohe], axis=1)
T_processed = pd.concat([T_num, T_cat_ohe], axis=1)

# sanitize for LightGBM
X_processed = clean_feature_names(X_processed)
T_processed = clean_feature_names(T_processed)

# align
T_processed = T_processed.reindex(columns=X_processed.columns, fill_value=0)

print("X_processed:", X_processed.shape, "T_processed:", T_processed.shape)

X_processed: (307511, 294) T_processed: (48744, 294)


In [11]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

for fold, (tr_idx, va_idx) in enumerate(folds.split(X_processed, y), 1):
    X_tr, X_va = X_processed.iloc[tr_idx], X_processed.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    model = LGBMClassifier(
        n_estimators=3000,
        learning_rate=0.03,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )

    model.fit(X_tr, y_tr)

    preds = model.predict_proba(X_va)[:, 1]
    auc = roc_auc_score(y_va, preds)
    scores.append(auc)
    print(f"Fold {fold} AUC: {auc:.5f}")

print("\nMean AUC:", float(np.mean(scores)))
print("Std AUC:", float(np.std(scores)))

Fold 1 AUC: 0.75805
Fold 2 AUC: 0.76085
Fold 3 AUC: 0.75605
Fold 4 AUC: 0.76418
Fold 5 AUC: 0.75732

Mean AUC: 0.7592903464746181
Std AUC: 0.0029076590862700093


# 5. Results & Comparison

## Expected outcome
* AUC should improve compared to Day 2 baseline (**0.7562**) because bureau features add historical behavior.

## Next steps
* Add `bureau_balance.csv` (Month-level bureau history)
* Add `previous_application.csv` (behavioral patterns in prior applications)
* Add `installments_payments.csv` (lateness and payment discipline)